<a href="https://colab.research.google.com/github/GaetanAm/Sentiment-Analysis-and-Impact-on-Financial-Markets/blob/main/notebooks/Stage1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

# Charger les données
file_path = "data/your_file.csv"  # Remplacez par le chemin de votre fichier
df = pd.read_csv(file_path)

# Aperçu des données
print(df.head())
print(df.info())
print(df.describe())

# Vérifier les valeurs manquantes
print(df.isnull().sum())


FileNotFoundError: [Errno 2] No such file or directory: 'data/your_file.csv'

In [ ]:
# Supprimer les lignes avec des valeurs manquantes
df = df.dropna()


In [ ]:
df['publishedAt'] = pd.to_datetime(df['publishedAt'])


In [ ]:
from sklearn.model_selection import train_test_split

# Diviser les données
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

print("Taille de l'ensemble d'entraînement :", len(train_data))
print("Taille de l'ensemble de test :", len(test_data))


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Convertir les titres en vecteurs numériques
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(train_data['title'])
X_test = vectorizer.transform(test_data['title'])

# Entraîner le modèle
model = LogisticRegression()
model.fit(X_train, train_data['sentiment'])

# Prédictions
predictions = model.predict(X_test)

# Évaluation
print(classification_report(test_data['sentiment'], predictions))


In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

conf_matrix = confusion_matrix(test_data['sentiment'], predictions)
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Prédictions')
plt.ylabel('Vraies valeurs')
plt.show()
